In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from time import time
from time import sleep
import itertools
from datetime import timedelta
import diffdist.functional as distops

import torch
import torchvision
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.autograd import Variable

import torch.nn.functional as F
from torchvision import datasets, transforms
from torch import nn, optim
import numpy as np
import os
import matplotlib.pyplot as plt
from time import time
from time import sleep
import itertools
from datetime import timedelta
import diffdist.functional as distops

import torch
import torchvision
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.autograd import Variable
import torch.distributed.rpc as rpc
from torch.distributed.optim import ZeroRedundancyOptimizer

In [ ]:
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
valset = datasets.MNIST('./data', download=True, train=False, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=512, shuffle=True)

In [ ]:
class MyConvModel(nn.Module):
    def __init__(self, ps, input_num_filters, output_num_filters, filter_size):
        super(MyConvModel, self).__init__()
        self.conv1 = nn.Conv2d(input_num_filters, output_num_filters, kernel_size=(filter_size, filter_size))
        self.pool = nn.MaxPool2d(2, 2)
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        return x

class MyOutputModel(nn.Module):
    def __init__(self, input_size, hidden_size, ouput_size):
        super(MyOutputModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, ouput_size)
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)
        
    def forward(self, x):
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
def _parameter_rrefs(module):
    param_rrefs = []
    for param in module.parameters():
        param_rrefs.append(RRef(param))
    return param_rrefs

class MyPipelineModel(nn.Module):
    def __init__(self, num_worker, split_size=32):
        super(MyPipelineModel, self).__init__()
        #self.models = models
        self.num_worker = num_worker
        self.split_size = split_size
        self.optimizer = ZeroRedundancyOptimizer(optim.SGD, self.parameter_rrefs(), lr=0.001,)
        self.criterion = nn.CrossEntropyLoss()
        self.epochs = 5
        #self.batch_size = 100
        self.running_loss = 0
        
        self.model1 = rpc.remote(trainee0, MyConvModel, args=(1, 16, 5))
        self.model2 = rpc.remote(trainee1, MyConvModel, args=(16, 32, 5))
        self.model3 = MyOutputModel(512, 128, 10)
        
    def run(self):
        processes = []
        
        for rank in range(self.num_worker):
            p = mp.Process(target=self.init_process, args=(rank, self.num_worker))
            p.start()
            processes.append(p)
        for p in processes:
            p.join()
            
    def parameter_rrefs(self):
        remote_params = []
        remote_params.extend(_remote_method(_parameter_rrefs, self.model1))
        remote_params.extend(_remote_method(_parameter_rrefs, self.model2))
        remote_params.extend(_parameter_rrefs(self.model3))
        
        return remote_params
        
    def init_process(rank, world_size):
        os.environ['MASTER_ADDR'] = 'localhost'
        os.environ['MASTER_PORT'] = '29500'
        if rank == world_size - 1:
            rpc.init_rpc("trainer", rank=rank, world_size=world_size)
            self.train()
        else:
            rpc.init_rpc("trainee" + str(rank), rank=rank, world_size=world_size)
            # parameter server do nothing
            pass

        # block until all rpcs finish
        rpc.shutdown()
        
    def forward(self, input):
        output1 = _remote_method(MyConvModel.forward, self.model1, input)
        output2 = _remote_method(MyConvModel.forward, self.model2, output1)
        return self.model3(output2)
        
    def train(self, rank):
        x = trainloader
        time0 = time()
        for e in range(self.epochs):
            self.running_loss = 0
            
            for images, labels in x:
                with dist_autograd.context() as context_id:
                    output = self(imgages)
                    loss = self.criterion(output, labels)
                    dist_autograd.backward(context_id, [loss])
                    slef.optimizer.step(context_id)
                    self.running_loss += loss.item()
            
            if rank == self.num_worker-1:
                print("Epoch {} - Training loss: {}".format(e, self.running_loss/len(trainloader)))
                print("\nTraining Time (in seconds) =",(time()-time0))

In [ ]:
model = MyPipelineModel(3)
model.run()